<div class="alert alert-block alert-info">
<b>V3 Комментарий ревьюера:</b>

Привет! Можешь не подбирать гиперпараметры у LightGBM. Только для RandomForestRegressor

Добавь пожалуйста анализ моделей (желательно все метрики занести в общую таблицу и показать), выбери одну модель и проведи на ней финальное тестирование.

Удачи!)
</div>

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Привет! Вторичные комментарии начинаю с V2

Я исправил разбиение на выборки и кодирование признаков.

Можешь обучить модели на новых выборках?) В анализе нужно учитывать как время обучения, так и время предсказания.

Я там оставил подсказки. Удачи!
</div>

Привет! Как дела?) ☺️
<br> Меня зовут Дуолан 👋 Буду проверять твой проект. Давай вместе доведем его до идеала 😉
<br> Дальнейшее общение будет происходить на «ты», если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий («исправил», «не понятно как исправить ошибку», ...)
<br> Пожалуйста, не удаляй мои комментарии, они будут необходимы для повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b> Если требуются исправления
</div>

Работа не может быть принята с красными комментариями.

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-info">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Нам нужно построить модель для определения рыночной стоимости автомобиля.\
Шаги:\
1)Загрузим данные.\
2)Изучим данные. Заполним пропущенные значения и обработаем аномалии в столбцах.\
3)Подготовим выборки для обучения моделей.\
4)Обучим разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. \
5)Проанализируем качество моделей.\
6)Опираясь на критерии заказчика, выберем лучшую модель, проверим её качество на тестовой выборке.

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b> 

Важно чтобы в проектах было подробное описание: когда потенциальные работодатели будут смотреть твой проект в портфолио, им нужно будет понимать задачу, чтобы можно было оценить её решение. Вот что можно добавить:
    
* Описание целей и задачи проекта
* Описание данных
* План работы

И рекомендую писать введение от первого лица множественного числа. Так работа выглядит более самостоятельной)
</div>

## Подготовка данных

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import re


Загрузим наши данные и проведем первичный анализ.

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b>

Можешь все импорты занести в эту ячейку?)
</div>

In [2]:
data = pd.read_csv('C://Users/gadji/Проекты/Autos/autos.csv')
display(data.head(10))

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [3]:
# Преобразование названий столбцов в snake_case
data.columns = [''.join(['_'+ c.lower() if c.isupper() else c for c in col]).lstrip('_') for col in data.columns]
data.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')

В наших данных целевая переменая - price(цена автомобиля). Как мы можем заметить, имеются пропуски в VehicleType(тип автомобиля), Model(модель автомобиля), и в признаке Repaired("битый" автомобиль).
Кроме того, для признака power(мощность автомобиля) есть странное значение 0, у автомобиля не может быть нулевой мощности.
Исследуем наши данные подробнее на пропуски, дубликаты и выбросы(аномалии).

In [4]:
data.isna().sum() / data.count() * 100

date_crawled           0.000000
price                  0.000000
vehicle_type          11.831014
registration_year      0.000000
gearbox                5.928510
power                  0.000000
model                  5.887995
kilometer              0.000000
registration_month     0.000000
fuel_type             10.232554
brand                  0.000000
repaired              25.123669
date_created           0.000000
number_of_pictures     0.000000
postal_code            0.000000
last_seen              0.000000
dtype: float64

В наших данных много пропусков (vehicle_type, gearbox, model, fuel_type, repaired), чтобы не потерять информативности заполним их.\
Рассмотрим признак VehicleType подробнее.

In [5]:
data['vehicle_type'].value_counts()

vehicle_type
sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: count, dtype: int64

Как мы можем заметить, разница между первыми 3 позициями не сильно большая, а значит заполнить пропуски наиболее популярными будет неверным решением. Пропуски в данном случае неочевидны. Тогда заполним их значением заглушкой "unknown".\
Так же поступим и с пропусками для признака model.

In [6]:
data[['model', 'vehicle_type']] = data[['model', 'vehicle_type']].fillna('unknown')

Пропуски в оставшихся признаках скорее всего подзразумевают стандартное, очевидное заполнение.\
Так, для признака gearbox - это механическая коробка передач(manual). Большинство машин на механической коробке передач.\ 
Для признака fuel_type - 'fueltype'. Большинство машин выпускают изначально на бензиновом топливе.\
Для признака not_repaired - no. Скорее всего, пропуск означает, что машина не попадала в аварии.

In [7]:
data['gearbox'] = data['gearbox'].fillna('manual')
data['fuel_type'] = data['fuel_type'].fillna('petrol')
data['repaired'] = data['repaired'].fillna('no')

In [8]:
data

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,no,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,unknown,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,unknown,2005,manual,0,unknown,20000,1,petrol,sonstige_autos,no,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [ ]:
print(data['registrationyear'].unique())
print(data['power'].sort_values(ascending=False).unique())
data = data.query('registrationyear < 2017 and registrationyear > 1960')
data = data.drop_duplicates().reset_index(drop=True)
data = data.query('power > 50 and power < 2028')

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> Как мы можем заметить в столбцах power и registrationyear есть аномалии избавимся от них. Год регистрации ограничим снизу 1960 годом а сверзу 2016.Мощность снизу 50 а сверху 2028(максимальная мощность машины на данный момент).
</div>

In [ ]:
print(data['price'].sort_values(ascending=False).unique())
data = data.query('price > 150')

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> Удалим  значения меньшие 150 в столбце price,ведь цена ниже может быть только на металлолом.
</div>

In [ ]:
# код ревьюера

data.info()

In [ ]:
features = data.drop(['price','datecrawled','datecreated','lastseen','postalcode','numberofpictures','registrationmonth'],axis=1)
target = data['price']

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Удалить неинформативные признаки - хорошее решение 👍
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b>

Месяц регистрации тоже не относится к важным признакам
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b>

Стоит проверить данные на наличие дубликатов
</div>

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b>

Нужно проверить данные на аномалии.

1. Год регистрации нужно ограничить сверху 2016 годом. Максимальный год — дата скачивания анкеты 2016.
2. В признаке 'power' есть выбросы и аномалии. Мощность можно ограничить максимальной и минимальной приемлемыми значениями
3. Присутствуют нулевые значения в признаке price. Цена не может быть нулевой, к тому же это целевой признак. Самое интересное, что на нескольких сайтах по продаже подержанных автомобилей в Германии (страну можно определить по почтовым индексам, большинство из них из Германии) показало, что цены, действительно, начинаются с 0 и 1 евро, но это единичные объявления. Не понятно, то ли это ошибка при заполнении формы, то ли машины на металлолом. Но я все-таки склоняюсь к тому, чтобы избавиться от таких данных.
</div>

<div class="alert alert-block alert-success">
<b>V2 ✔️ Успешно исправлено</b>
</div>

## Обучение моделей

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Для моделей, основанных на деревьях решений, порядковое кодирование отлично подойдет 👍
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b>

Кодировать данные стоит после разбиения. Так у нас не будет утечки данных.
</div>

In [ ]:
features_train, features_valid,target_train,target_valid = train_test_split(features,target,test_size=0.4,random_state=12345)
features_valid,features_test,target_valid,target_test = train_test_split(features_valid,target_valid,test_size = 0.5,random_state=12345)

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Данные поделены верно, пропорции выбраны адекватные 👍
</div>

In [ ]:
display(features_train)

In [ ]:
# код ревьюера

features_train_ohe = pd.get_dummies(features_train, drop_first=True)
features_valid_ohe = pd.get_dummies(features_valid, drop_first=True)
features_test_ohe = pd.get_dummies(features_test, drop_first=True)

In [ ]:
# код ревьюера

features_train_ohe.shape, features_valid_ohe.shape, features_test_ohe.shape

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

На валидационной выборке получилось на один признак меньше. Сейчас проведу проверку на лишние признаки и удалю их
</div>

In [ ]:
# код ревьюера

for col in features_train_ohe.columns.to_list():
    if col not in features_valid_ohe.columns.to_list():
        print(col)

In [ ]:
# код ревьюера

del features_train_ohe['model_kalina']
del features_train_ohe['model_serie_2']
del features_train_ohe['brand_trabant']

In [ ]:
# код ревьюера

for col in features_train_ohe.columns.to_list():
    if col not in features_test_ohe.columns.to_list():
        print(col)

In [ ]:
# код ревьюера

del features_train_ohe['model_samara']
del features_train_ohe['model_serie_3']

In [ ]:
# код ревьюера

for col in features_valid_ohe.columns.to_list():
    if col not in features_train_ohe.columns.to_list():
        print(col)

In [ ]:
# код ревьюера

del features_valid_ohe['model_range_rover_evoque']
del features_valid_ohe['model_rangerover']
del features_valid_ohe['model_samara']
del features_valid_ohe['model_serie_3']

In [ ]:
# код ревьюера

for col in features_valid_ohe.columns.to_list():
    if col not in features_test_ohe.columns.to_list():
        print(col)

In [ ]:
# код ревьюера

for col in features_test_ohe.columns.to_list():
    if col not in features_train_ohe.columns.to_list():
        print(col)

In [ ]:
# код ревьюера

del features_test_ohe['model_kalina']
del features_test_ohe['model_range_rover_evoque']
del features_test_ohe['model_rangerover']
del features_test_ohe['model_serie_2']
del features_test_ohe['brand_trabant']

In [ ]:
# код ревьюера

for col in features_test_ohe.columns.to_list():
    if col not in features_valid_ohe.columns.to_list():
        print(col)

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Теперь нужно стандартизировать численные признаки
</div>

In [ ]:
features_train_ohe.head()

In [ ]:
scaler = StandardScaler()
numeric = ['registrationyear', 'power', 'kilometer']

scaler.fit(features_train_ohe[numeric])

features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_valid_ohe[numeric] = scaler.transform(features_valid_ohe[numeric])
features_test_ohe[numeric] = scaler.transform(features_test_ohe[numeric])

In [ ]:
features_train_ohe.head()

In [ ]:
#features_train = pd.get_dummies(features_train,drop_first=True)
#features_valid = pd.get_dummies(features_valid,drop_first=True)
#features_test = pd.get_dummies(features_test,drop_first=True)
#object = StandardScaler()
#features_train1 = object.fit_transform(features_train)
#features_valid1 = object.fit_transform(features_valid)
#features_test1 = object.fit_transform(features_test)

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> Когда примения стандартизацию получается вектор, а не таблица и при предсказании модели выдает ошибку(matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 304 is different from 303))
</div>

In [ ]:
#model = LinearRegression()
#model.fit(features_train,target_train)
#predictions = model.predict(features_valid)
#print(mean_squared_error(target_valid,predictions) ** 0.5)

In [ ]:
#features_train, features_valid,target_train,target_valid = train_test_split(features,target,test_size=0.4,random_state=12345)
#features_valid,features_test,target_valid,target_test = train_test_split(features_valid,target_valid,test_size = 0.5,random_state=12345)

In [ ]:
#encoder = OrdinalEncoder()
#def encod(features):
#    features = pd.DataFrame(encoder.fit_transform(features),columns=features.columns)
#    features = features.astype('int')
#    return features

In [ ]:
#features_train = encod(features_train)
#features_valid = encod(features_valid)
#features_test = encod(features_test)

In [ ]:
#features_train

In [ ]:
# код ревьюера

features_train.head()

In [ ]:
# код ревьюера

encoder = OrdinalEncoder()
cat_col = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']

features_train_oe = features_train.copy()
features_valid_oe = features_valid.copy()
features_test_oe = features_test.copy()

features_train_oe[cat_col] = encoder.fit_transform(features_train_oe[cat_col])
features_valid_oe[cat_col] = encoder.fit_transform(features_valid_oe[cat_col])
features_test_oe[cat_col] = encoder.fit_transform(features_test_oe[cat_col])

In [ ]:
# код ревьюера

features_train_oe.head()

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Данные готовы для обучения:

features_train_ohe, features_valid_ohe, features_test_ohe для линейной регресии
    
features_train_oe, features_valid_oe, features_test_oe для бустингов и деревьев решений
</div>

# Обучение моделей

## Линейная регрессия

In [ ]:
# код ревьюера

model = LinearRegression()

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Время обучения
</div>

In [ ]:
%%time

# код ревьюера

model.fit(features_train_ohe, target_train)

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Время предсказания
</div>

In [ ]:
%%time

# код ревьюера

pred = model.predict(features_valid_ohe)

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Метрика на валидационной выборке
</div>

In [ ]:
print('Linear Regression RMSE:', mean_squared_error(target_valid, pred, squared=False))

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Можешь выполнить такое и для других моделей?)

Но для LightGBM и RandomForestRegressor сначала нужно подобрать гиперпараметры
</div>

## LightGBM

In [ ]:
#%%time
#model = LGBMRegressor(num_iterations=200,max_depth=9)
#model.fit(features_train,target_train)

In [ ]:
#%%time
#predictions = model.predict(features_valid)

In [ ]:
#print(mean_squared_error(target_valid,predictions) ** 0.5)

In [ ]:
#инициализация модели
model = LGBMRegressor(max_depth=10)

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> Спасибо за подсказки! Пытаюсь подобрать гипермараметры.И не понимаю почему модель так долго обучается.Чем чаще я перезапускаю ноутбук тем дольше она обучается.Еще вчера все работало.Сегодня уже не могу дождаться конца обучения при подборе.Код такой же а время с каждым разом увеличивается.Не могу понять в чем дело.
</div>

<div class="alert alert-block alert-info">
<b>V3 Комментарий ревьюера:</b>

Да, есть такое) Но такое происходит только у LightGBM. Поэтому у этой модели можешь не подбирать гиперпараметры.
</div>

In [ ]:
#подбор гиперпараметров и рассчет метрики

In [ ]:
%%time
model.fit(features_train_oe,target_train)

In [ ]:
%%time

#время предсказания

predictions = model.predict(features_valid_oe)

In [ ]:
print('LGBMRegerssor RMSE :', mean_squared_error(target_valid,predictions,squared=False))

<div class="alert alert-block alert-success">
<b>V3 ✔️ Успех:</b>

Модель обучена верно 👍
</div>

## RandomForestRegressor

In [ ]:
#rf_grid = [{'n_estimators':[2,50],'max_depth':[1,10]}]
#forest_reg = RandomForestClassifier()
#model = GridSearchCV(forest_reg, rf_grid,
#                         scoring = mean_squared_error, cv=2, verbose=1)
#model.fit(features_train,target_train)
#predictions = model.predict(features_valid)
#print(model.best_params_)
#print()
#print(mean_squared_error(predictions,target_valid)**0.5)

In [ ]:
#%time
#model = RandomForestClassifier(n_estimators =1 , max_depth = 2, random_state=12345)
#model.fit(features_train,target_train)
#predictions = model.predict(features_valid)
#print(mean_squared_error(predictions,target_valid) ** 0.5)

In [ ]:
min = 10000
for depth in range(1,5):
    model = RandomForestClassifier(max_depth=depth)
    model.fit(features_train_oe,target_train)
    predictions = model.predict(features_valid_oe)
    mse =  mean_squared_error(target_valid,predictions,squared=False)
    if min > mse:
        min = mse 
        d = depth
print(min,depth)

In [ ]:
#инициализация модели
min = 10000
for n in [10,20,30,40,50]:
    model = RandomForestClassifier(n_estimators=n,max_depth=depth)
    model.fit(features_train_oe,target_train)
    predictions = model.predict(features_valid_oe)
    mse =  mean_squared_error(target_valid,predictions,squared=False)
    if min > mse:
        min = mse 
        num = n
print(min,num)

In [ ]:
#подбор гиперпараметров и рассчет метрики
model = RandomForestClassifier(n_estimators=num,max_depth=depth)


In [ ]:
%%time
#время обучения
model.fit(features_train_oe,target_train)

In [ ]:
%%time
#время предсказания
predictions = model.predict(features_valid_oe)

In [ ]:
print('RMSE RandomForesClassifier:', mean_squared_error(target_valid,predictions,squared=False))

<div class="alert alert-block alert-success">
<b>V3 ✔️ Успех:</b>

Модель обучена верно 👍
</div>

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b>

Можешь добавить обучение еще одной простой модели?) Например, Случайный лес
</div>

<div class="alert alert-block alert-success">
<b>V2 ✔️ Успешно исправлено</b>
</div>

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b>

Стоит добавить оптимизацию (подбор гиперпараметров) хотя бы для одной модели.
</div>

<div class="alert alert-block alert-success">
<b>V2 ✔️ Успешно исправлено</b>
</div>

# Анализ моделей

Таким образом, модель с лучшей метрикой равной 1747, что меньше 2500 это LightGBM модель.Проверим ее на тестовой выборке.

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

В анализе нужно учитывать время обучения, время предсказания и RMSE
</div>

LinearRegression : время обучения - 12.6 с , время предсказания 118 мс, RMSE равно 2671.
LGBMRegressor : время обучения - 6.65с, время предсказания  - 402мс , RMSE равно  1708.
RandomForestClassifier : время обучения - 3.49с , время предсказания - 40с  , RMSE равно 3133.

<div class="alert alert-block alert-info">
<b>V3 Комментарий студента:</b>
LinearRegression : время обучения - 12.6 с , время предсказания 118 мс, RMSE равно 2671.<br>
LGBMRegressor : время обучения - 6.65с, время предсказания  - 402мс , RMSE равно  1708.<br>
RandomForestClassifier : время обучения - 3.49с , время предсказания - 40с  , RMSE равно 3133.
</div>

<div class="alert alert-block alert-info">
<b>V3 Комментарий студента:</b>Таким образом, Наилучшая модель с наименьшим RMSE это LBGMRegressor.
</div>

# Тестирование лучшей модели

In [ ]:
model = LGBMRegressor(max_depth=10)
model.fit(features_train_oe, target_train)
predictions = model.predict(features_test_oe)
print(mean_squared_error(target_test,predictions,squared = False))

<div class="alert alert-block alert-info">
<b>V3 Комментарий студента:</b>Модель прошла тестирование.
</div>

<div class="alert alert-block alert-success">
<b>V4 ✔️ Успешно исправлено</b>
</div>

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> Не понял почему у меня модель бустинга стала так долго работать (10 мин притом что я уменьшил количство итераций с 500 до 200). Не работает линейная регрессия, Не могу разобраться с прямым кодированием данных.
</div>

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Я помогу тебе с подготовкой данных)

Оставлю подсказки по обучению и анализу моделей
</div>

Как мы видим метрика меньше 2500 а значит модель нам подходит.

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b>

В сравнении нужно смотреть отдельно на время обучения и время предсказания для каждой модели.

Анализ моделей должен выглядеть так:

1. RMSE вычисляется на валидационной выборке
2. Время обучения = время model.fit(X_train, y_train)
3. Время предсказания = model.predict(X_valid)
</div>

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b>

После анализа моделей нужно порекомендовать заказчику одну модель и провести на ней финальное тестирование.

Рассчитай финальную метрику лучшей модели на тестовой выборке (до этого тестовая выборка нигде не должна использоваться). RMSE должно быть меньше 2500. Если метрика не дотягивает, попробуй исправить мои замечания, также можешь потюнить гиперпараметры (на этапе обучения моделей, не на тестовой выборке!)
</div>

<div class="alert alert-block alert-info">
<b>Комментарий ревьюера:</b>

Чтобы все было ясно, напишу конкретный план (структуру) обучения и анализа моделей:

1. Обучение моделей. В обучении нужно рассмотреть хотя бы одну простую модель и один бустинг. Подбор гиперпараметров нужно провести хотя бы одной модели. Тут есть два варианта:
    - без валидационной выборки. Здесь нужно подбирать гиперпараметры с помощью кросс-валидации (GridSearchCV, RandomizedSearchCV или вручную (cross_val_score));
    - валидационная выборка есть. Здесь можно не использовать кросс-валидацию и подбирать гиперпараметры вручную.  
2. Анализ моделей. После нахождения лучших гиперпараметров стоит измерить время обучения, предсказания и RMSE. Тут тоже есть два варианта:
    - без валидационной выборки. RMSE на кросс-валидации. Время обучения = время model.fit(X_train). Время предсказания  = model.predict(X_train);
    - валидационная выборка есть. RMSE на validation. Время обучения = время model.fit(X_train). Время предсказания = время model.predict(X_valid).  
    
    После этого напиши вывод по анализу (можешь также результаты моделей занести в общую таблицу) и посоветуй заказчику одну модель на основе его критериев;
3. Тестирование. Рассчитай финальную метрику лучшей модели на тестовой выборке (до этого тестовая выборка нигде не должна использоваться!). RMSE должно быть меньше 2500. Если метрика не дотягивает, попробуй исправить мои замечания, также можешь потюнить гиперпараметры (на этапе обучения моделей, не на тестовой выборке!)
</div>

# <font color='orange'>Общее впечатление</font>
* Молодец, что структурируешь ноутбук, приятно проверять такие работы
* У тебя чистый и лаконичный код
* Исправь, пожалуйста, мои замечания. Затем отправляй на повторную проверку
* Жду новую версию проекта 👋

# <font color='orange'>2. Общее впечатление</font>
* Спасибо за быстрое внесение правок
* Теперь проект выглядит лучше )
* Критических замечаний нет
* Молодец, отличная работа!
* Надеюсь, ревью было полезным
* Удачи в дальнейшем обучении 👋

# <font color='orange'>Рекомендации 🔥</font>
* Доступное объяснение про работу градиентного бустинга https://www.youtube.com/watch?v=ZNJ3lKyI-EY&t=995s
* После просмотра видео можешь почитать статью про ансамбли и бустинги https://habr.com/ru/company/ods/blog/645887/
* Подбор гиперпараметров для CatBoost https://habr.com/ru/company/otus/blog/527554/
* Подбор гиперпараметров для LightGBM https://programmersought.com/article/40776531516/
* Разумные способы кодирования категориальных данных для машинного обучения https://machinelearningmastery.ru/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159/
* Статья про "непонятную" метрику ROC-AUC https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
* Хочешь подтянуть математику для DS?) https://academy.stepik.org/math-for-data-science
* Результы обучения моделей можно визуализировать 😎 https://www.datatechnotes.com/2019/08/elasticnet-regression-example-in-python.html
* В нашем деле нужно быть всегда в курсе всех новинок, новостей и тд, вот тут публикуют новости в области DS: https://www.infoq.com/data-analysis/news/
* Всем аналитикам данных рекомендую книгу Даниела Канемана "Думай медленно, решай быстро"